In [4]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'

db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [10]:
query = '''
SELECT 
    *
FROM
    document_mst AS dm
        INNER JOIN
    account_mst AS am ON dm.account_id = am.id
WHERE
    am.facility_id = 168
        AND dm.is_physician_document = 1
        AND dm.service_date >= '2023-05-01';
'''

In [11]:
df_query = pd.read_sql(query, con=db_connection)

In [16]:
name_list = df_query.name.tolist()

In [17]:
name_list

['637523808',
 '637523821',
 '637523833',
 '637523849',
 '639701306',
 '637523880',
 '637523956',
 '637523963',
 '639701322',
 '637523971',
 '637523980',
 '637523983',
 '637523994',
 '637524002',
 '637524007',
 '637524016',
 '637524038',
 '637524022',
 '637524046',
 '637524052',
 '637594025',
 '638692545',
 '637594032',
 '637594106',
 '637594124',
 '637594227',
 '637594188',
 '637594248',
 '637594266',
 '637594046',
 '637594057',
 '638585993',
 '639701335',
 '637594152',
 '637594293',
 '637594318',
 '637594342',
 '637594329',
 '637594371',
 '637594385',
 '637594398',
 '637594409',
 '639701291',
 '637594424',
 '637594456',
 '637594484',
 '640922759',
 '637594538',
 '637594556',
 '637594569',
 '637594597',
 '637594608',
 '637594623',
 '637594658',
 '638585999',
 '639701274',
 '637594671',
 '637594684',
 '637594734',
 '637594748',
 '637594814',
 '637594801',
 '639701367',
 '637594833',
 '637594845',
 '637594860',
 '637594895',
 '637594918',
 '637594933',
 '637594951',
 '639701388',
 '6375

In [26]:
import os
import shutil

source_folder = r"/home/trishalaswain/Documents/analysis/data/download_scripts_and_tars/OCR_PROJECT_TEST/PDF/"
# destination_folder = r"/home/trishalaswain/Documents/analysis/data/download_scripts_and_tars/OCR_PROJECT_TEST/PDF/"

for sub1 in os.listdir(source_folder):
    # pdf_name = sub1.replace(".json", "")
    pdf_name = sub1.replace(".pdf", "")

    # print(pdf_name.tr)
    # if pdf_name[1:] not in name_list:
    if pdf_name not in name_list:
        # shutil.move(source_folder +sub1, destination_folder+sub1)
        os.remove(source_folder +sub1)

In [8]:
df_query.columns

Index(['id', 'account_id', 'name', 'path', 'service_date', 'document_date',
       'event_time', 'physician_id', 'format', 'type_id', 'status',
       'is_physician_document', 'orignal_format', 'orignal_path',
       'message_s3_path', 'character_count', 'page_count', 'size_in_byte',
       'original_document_page_count', 'original_document_size_in_byte',
       'is_active', 'created_by', 'updated_by', 'created_date', 'updated_date',
       'red_dp_ts', 'id', 'client_id', 'facility_id', 'facility_type', 'mrn',
       'facility_mrn', 'account_number', 'facility_account_number',
       'patient_first_name', 'patient_last_name', 'patient_middle_name',
       'birth_datetime', 'gender', 'admission_datetime', 'discharge_datetime',
       'previous_service_datetime', 'event_time', 'mpn', 'building', 'floor',
       'room', 'bed', 'point_of_care_id', 'patient_class_id',
       'patient_type_id', 'service_line_id', 'financial_class_id', 'payor_id',
       'payor_group_id', 'admit_source_id', '

In [9]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |    0 |
|--------------:|-----:|
|           168 | 1343 |


In [12]:
for name, path in zip(df_query.name, df_query.path):
    if path == None or name ==None:
        continue

# fac_name=file_path.split("/")[2]
# cli=file_path.split("/")[0]
# fac_list.append(fac_name)

# original path
    print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}.pdf".format(env,path,name))

aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523806_20230509164317.017.pdf' 637523806.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523807_20230509164315.015.pdf' 637523807.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523808_20230509164314.014.pdf' 637523808.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523809_20230509164316.016.pdf' 637523809.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523811_20230509164315.015.pdf' 637523811.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523814_20230509164314.014.pdf' 637523814.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523815_20230509164312.012.pdf' 637523815.pdf
aws s3 cp 's3://ezdi-production-bucket/ezcapc/SCP/data/6494/document/20230501/637523816_20230509164321.021.pdf' 637523